# Лабораторная работ №3 - "Подбор гиперпараметров модели"

---

#### Выполнил: Книга Тимофей
#### Группа: М80-309Б-23
#### Датасет (ссылка): [Факторы успеваемости студентов](https://www.kaggle.com/datasets/lainguyn123/student-performance-factors)

---

# Задание:

0. Выбрать модель для обучения (Desicion tree, Random forest, SVM, KNN, Boosting)
1. Показать какие гиперпараметры есть у выбранной модели (В виде таблички).
2. Выбрать датасет для обучения и взависимости от модели подготовить данные
3. Подобрать гиперпараметры для модели и сравнить лучшие подборы, для (Grid Search, RandomSearch, Optuna)
4. На самом лучшем обучении (Grid Search, RandomSearch, Optuna) сделать калькулятор, который показывает `локальную интерпретацию` с помощью [LIME](https://habr.com/ru/companies/otus/articles/779430/) и `глобальную интерпретацию` с помощью [SHAP](https://habr.com/ru/companies/ods/articles/599573/).

    [Доп статья](https://habr.com/ru/companies/wunderfund/articles/739744/)

Загрузим датасет

In [ ]:
import pandas as pd

df = pd.read_csv('~/University/ML/StudentPerformanceFactors.csv')

df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Hours_Studied               6607 non-null   int64 
 1   Attendance                  6607 non-null   int64 
 2   Parental_Involvement        6607 non-null   object
 3   Access_to_Resources         6607 non-null   object
 4   Extracurricular_Activities  6607 non-null   object
 5   Sleep_Hours                 6607 non-null   int64 
 6   Previous_Scores             6607 non-null   int64 
 7   Motivation_Level            6607 non-null   object
 8   Internet_Access             6607 non-null   object
 9   Tutoring_Sessions           6607 non-null   int64 
 10  Family_Income               6607 non-null   object
 11  Teacher_Quality             6529 non-null   object
 12  School_Type                 6607 non-null   object
 13  Peer_Influence              6607 non-null   obje

Вот подробное описание параметров:

| Признак | Перевод | Возможные значения |
|--------|---------|--------------------|
| **Hours_Studied** | Количество часов учёбы | Число |
| **Attendance** | Посещаемость (%) | Число 0–100 |
| **Parental_Involvement** | Вовлечённость родителей | Low (низкая), Medium (средняя), High (высокая) |
| **Access_to_Resources** | Доступ к учебным ресурсам | Low (низкий), Medium (средний), High (высокий) |
| **Extracurricular_Activities** | Внеучебная активность | Yes (да), No (нет) |
| **Sleep_Hours** | Количество часов сна | Число |
| **Previous_Scores** | Предыдущие оценки | Число |
| **Motivation_Level** | Уровень мотивации | Low (низкая), Medium (средняя), High (высокая) |
| **Internet_Access** | Доступ к интернету | Yes (да), No (нет) |
| **Tutoring_Sessions** | Количество занятий с репетитором | Число |
| **Family_Income** | Доход семьи | Low (низкий), Medium (средний), High (высокий) |
| **Teacher_Quality** | Качество преподавателя | Low (низкое), Medium (среднее), High (высокое) |
| **School_Type** | Тип школы | Public (государственная), Private (частная) |
| **Peer_Influence** | Влияние сверстников | Positive (положительное), Neutral (нейтральное), Negative (отрицательное) |
| **Physical_Activity** | Физическая активность (часы/неделя) | Число |
| **Learning_Disabilities** | Проблемы с обучением | Yes (да), No (нет) |
| **Parental_Education_Level** | Образование родителей | High School (средняя школа), College (колледж), Postgraduate (высшее+) |
| **Distance_from_Home** | Расстояние от дома | Near (близко), Moderate (средне), Far (далеко) |
| **Gender** | Пол | Male (мужской), Female (женский) |
| **Exam_Score** | Итоговый балл | Число |


Так как в нашем датасете достаточно много категориальных признаков, модель просто не сможет с ними правильно работать. Чтобы не создавать ложный порядок элементов, применив LabelEncoding, применим OneHotEncoding для категориальных признаков.

In [3]:
df = pd.get_dummies(df, drop_first=True)

Делим на test и train

In [4]:
from sklearn.model_selection import train_test_split
X, y = df.drop('Exam_Score', axis=1), df['Exam_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Создаем пайплайн

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

# Пайплайн с масштабированием
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeRegressor(random_state=42))  # Это задача регресии: определить количество баллов за экзамен
])

# Просмотр пайплайна
pipeline

,steps,"[('scaler', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,2


Параметры для перебора для методов:

* Grid Search
* Random Search

In [6]:
# Параметры
param_grid = {
    "classifier__max_depth": [3, 4, 5, 6, 7, None],
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__min_samples_leaf": [1, 2, 4],
    "classifier__max_features": ["sqrt", "log2", None]
}

# Grid Search
- Перебирает все возможные комбинации указанных гиперпараметров.
- Для каждой комбинации обучает модель и проверяет качество на кросс-валидации.
- Возвращает лучшую комбинацию по заданной метрике (например, accuracy).

In [7]:
from sklearn.model_selection import GridSearchCV

# Создание сетки
gs_tree = GridSearchCV(
    estimator=pipeline,                 # модель
    param_grid=param_grid,              # параметры
    scoring='neg_mean_squared_error',   # метрика
    cv=5,                               # K-folds для кросс валидации
    n_jobs=-1,                          # параллель на всю катушку
    verbose=3,                          # кол-во информации для отображения
)

# Обучение моделей
gs_tree.fit(X_train, y_train)

# Результаты
print("\nЛучшие параметры:")
for key, value in gs_tree.best_params_.items():
    print(f'{key} -> {value}')
print("\nЛучшее качество на кросс-валидации:", gs_tree.best_score_)
print("R2 на тесте:", gs_tree.score(X_test, y_test))

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV 3/5] END classifier__max_depth=3, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2;, score=-11.299 total time=   0.0s
[CV 2/5] END classifier__max_depth=3, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2;, score=-9.461 total time=   0.0s
[CV 1/5] END classifier__max_depth=3, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=5;, score=-9.256 total time=   0.0s
[CV 2/5] END classifier__max_depth=3, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=5;, score=-9.461 total time=   0.0s
[CV 1/5] END classifier__max_depth=3, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2;, score=-9.256 total time=   0.0s
[CV 5/5] END classifier__max_depth=3, classifier__max_features=sqrt, classifier__min_samples_leaf=1, 

# Random Search

* **Идея:** выбираем случайные комбинации гиперпараметров из заранее определённых диапазонов.
* **Плюсы:**
  * Прост в реализации.
  * Эффективнее, чем Grid Search для моделей с большим числом гиперпараметров.
  * Хорош для грубой оценки влияния гиперпараметров.
* **Минусы:**
  * Не гарантирует нахождение оптимального решения.
  * Требует большого числа итераций при сложных пространствах гиперпараметров.


In [8]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=pipeline,                 # модель
    param_distributions=param_grid,     # параметры
    n_iter=40,                          # кол-во попыток (кандидатов)
    scoring='neg_mean_squared_error',   # метрика
    cv=5,                               # K-folds для кросс валидации
    n_jobs=-1,                          # параллель на всю катушку
    verbose=3,                          # кол-во информации для отображения
)

random_search.fit(X_train, y_train)

# Результаты
print("\nЛучшие параметры:")
for key, value in random_search.best_params_.items():
    print(f'{key} -> {value}')
print("\nЛучшее качество на кросс-валидации:", random_search.best_score_)
print("R2 на тесте:", random_search.score(X_test, y_test))

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5] END classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=4, classifier__min_samples_split=10;, score=-7.715 total time=   0.0s
[CV 2/5] END classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=4, classifier__min_samples_split=10;, score=-8.231 total time=   0.0s
[CV 3/5] END classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=4, classifier__min_samples_split=10;, score=-9.390 total time=   0.0s
[CV 4/5] END classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=4, classifier__min_samples_split=10;, score=-9.879 total time=   0.0s
[CV 5/5] END classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf=4, classifier__min_samples_split=10;, score=-12.809 total time=   0.0s
[CV 1/5] END classifier__max_depth=4, classifier__max_features=log2, classifier__min_samples_leaf

# Байесовская оптимизация с Optuna

* **Идея:** строим вероятностную модель функции потерь и используем её, чтобы выбирать новые гиперпараметры «умнее», чем случайно.
* **Плюсы:**
  * Эффективнее Random Search при сложных пространствах гиперпараметров.
  * Балансирует *exploration* (исследование) и *exploitation* (использование лучших областей).
* **Минусы:**
  * Сложнее для понимания и настройки.
  * Требует больше зависимостей (Optuna, иногда PyTorch/Scikit-learn).

[Подробнее](https://optuna.org/)

In [9]:
# Установка Optuna
!pip install optuna

In [10]:
import optuna
from sklearn.model_selection import cross_val_score

def objective_dt(trial):
    # Настройка гиперпараметров для Decision Tree
    max_depth = trial.suggest_int('classifier__max_depth', 1, 20)
    min_samples_split = trial.suggest_int('classifier__min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('classifier__min_samples_leaf', 1, 10)

    pipeline.set_params(
        classifier__max_depth=max_depth,
        classifier__min_samples_split=min_samples_split,
        classifier__min_samples_leaf=min_samples_leaf,
    )

    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    return score


# Создаём исследование
study_dt = optuna.create_study(
    direction='maximize'  # максимизирукм cross_val_score чтобы он был наибольшим
)

study_dt.optimize(
    objective_dt,
    n_trials=60,  # количество попыток
    n_jobs=-1,
)


# Лучшие параметры
print("\nЛучшие параметры Decision Tree:")
for key, value in study_dt.best_params.items():
    print(f'{key} -> {value}')

pipeline.set_params(**study_dt.best_params)
pipeline.fit(X_train, y_train)
print("\nЛучшее качество на кросс-валидации:", cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean())
print("R2 Decision Tree на тесте:", pipeline.score(X_test, y_test))


/Users/nugget/University/ML/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-29 16:55:24,851] A new study created in memory with name: no-name-19572a26-a709-4e7e-a555-f06552889bbf
[I 2025-11-29 16:55:25,079] Trial 0 finished with value: -9.2933953665005 and parameters: {'classifier__max_depth': 8, 'classifier__min_samples_split': 8, 'classifier__min_samples_leaf': 7}. Best is trial 0 with value: -9.2933953665005.
[I 2025-11-29 16:55:25,087] Trial 2 finished with value: -9.2933953665005 and parameters: {'classifier__max_depth': 5, 'classifier__min_samples_split': 17, 'classifier__min_samples_leaf': 3}. Best is trial 0 with value: -9.2933953665005.
[I 2025-11-29 16:55:25,088] Trial 3 finished with value: -9.36634869787071 and parameters: {'classifier__max_depth': 16, 'classifier__min_sampl


Лучшие параметры Decision Tree:
classifier__max_depth -> 7
classifier__min_samples_split -> 5
classifier__min_samples_leaf -> 2

Лучшее качество на кросс-валидации: -8.799941673969165
R2 Decision Tree на тесте: 0.5408827988870817


# TPOT - Automated Machine Learning (AutoML) инструмент

TPOT (Tree-based Pipeline Optimization Tool) - это библиотека автоматизированного машинного обучения (AutoML), которая использует генетические алгоритмы для оптимизации полных конвейеров машинного обучения. В отличие от Optuna и RandomizedSearchCV, которые оптимизируют только гиперпараметры существующей модели, TPOT автоматически:


* Выбирает лучшие алгоритмы машинного обучения
* Создает оптимальные preprocessing steps
* Настраивает гиперпараметры
* Строит полные конвейеры обработки данных

## Ключевые особенности TPOT

### 1. **Генетическое программирование**
TPOT использует эволюционные алгоритмы для "размножения" и "мутации" конвейеров, постепенно улучшая их качество.

### 2. **Автоматический подбор моделей**
Может выбирать между различными классификаторами/регрессорами:
- Decision Trees, Random Forest, Gradient Boosting
- SVM, KNN, Logistic Regression
- Neural Networks и другие

### 3. **Предобработка данных**
Автоматически добавляет необходимые этапы:
- StandardScaler, MinMaxScaler, RobustScaler
- PCA, SelectPercentile, VarianceThreshold
- OneHotEncoding, PolynomialFeatures

## Преимущества TPOT

- **Экономия времени** - автоматизирует рутинные задачи
- **Обнаружение неочевидных решений** - может найти нестандартные комбинации методов
- **Воспроизводимость** - генерирует готовый Python код
- **Избегает переобучения** - использует кросс-валидацию

[Подробнее](https://github.com/EpistasisLab/tpot?ysclid=mhrqdj7la9508680297)

In [11]:
!pip install tpot

In [12]:
from tpot import TPOTClassifier
from sklearn.metrics import accuracy_score


# Создаём TPOT-классификатор
tpot = TPOTClassifier(
    generations=5,        # количество поколений эволюций
    population_size=20,   # размер популяции
    verbose=2,            # уровень детализации вывода
    random_state=42,
    cv=5,                 # количество фолдов кросс-валидации
)


/Users/nugget/University/ML/.venv/lib/python3.13/site-packages/stopit/__init__.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [13]:
# Обучаем
print("Начало оптимизации с TPOT...")
tpot.fit(X_train, y_train)


Начало оптимизации с TPOT...


/Users/nugget/University/ML/.venv/lib/python3.13/site-packages/tpot/tpot_estimator/estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
/Users/nugget/University/ML/.venv/lib/python3.13/site-packages/tpot/tpot_estimator/estimator.py:537: UserWarning: Labels are not encoded as ints from 0 to N. For compatibility with some classifiers such as sklearn, TPOT has encoded y with the sklearn LabelEncoder. When using pipelines outside the main TPOT estimator class, you can encode the labels with est.label_encoder_
  warnings.warn("Labels are not encoded as ints from 0 to N. For compatibility with some classifiers such as sklearn, TPOT has encoded y with the sklearn LabelEncoder. When using pipelines outside the main TPOT estimator class, you can encode the labels with est.label_encoder_")
Generation:   0%| 

,search_space,<tpot.search_...t 0x1770b1be0>
,scorers,['roc_auc_ovr']
,scorers_weights,[1]
,cv,5
,other_objective_functions,[]
,other_objective_functions_weights,[]
,objective_function_names,None
,bigger_is_better,True
,categorical_features,None
,memory,None
,preprocessing,False


In [14]:
# Оценка качества
y_pred = tpot.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Точность на тестовой выборке: {test_accuracy:.4f}")

# Лучшие параметры и конвейер
print("\nЛучший конвейер:")
print(tpot.fitted_pipeline_)

Точность на тестовой выборке: 0.0000

Лучший конвейер:
Pipeline(steps=[('robustscaler',
                 RobustScaler(quantile_range=(0.2475690452454,
                                              0.9671736154307))),
                ('selectpercentile',
                 SelectPercentile(percentile=97.7405997032442)),
                ('featureunion-1',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer()),
                                                ('passthrough',
                                                 Passthrough())])),
                ('featureunion-2',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer()),
                                                ('passthrough',
                                                 Passthrough())])),
                ('mlpclassifier',
                 MLPClassifier(activati

# Сравнение

| Метод | Что оптимизирует | Автоматизация | Время | Качество |
|-------|------------------|---------------|-------|----------|
| **Optuna** | Гиперпараметры модели | Средняя | Среднее | Высокое |
| **RandomizedSearchCV** | Гиперпараметры модели | Низкая | Быстрое | Среднее |
| **TPOT** | Полный конвейер + модели | Полная | Длительное | Очень высокое |